# Sentence Splitting
This notebook uses OCRed text for a all volume years in a folder and splits them into sentences using regular expression pattern matching.<br>
For this notebook to run, there should be an OCRed folder that should contain a .txt file, a .tsv file, and an images sub-folder (more details in the notebook).
<br>There should also be an accompanying `splitting_functs.py` file which has most of the functions required to run this notebook.

In [1]:
# from nltk.tokenize import PunktSentenceTokenizer
import nltk
import pandas as pd
import re
import sys
import multiprocessing

from tqdm import tqdm  # For printing out progress bar
from numpy import nan, array_split

from os import listdir, cpu_count

from splitting_functs import *

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_colwidth', None)

In [2]:
# Read all folder names in the OCR (or a specified) directory
# ocred_path = '/work/otb-lab/OCRed'
ocred_path = '/Users/nitingupta/Desktop/OTB/OCRed'

years = [name for name in listdir(ocred_path) if not name.startswith('.')]
years.sort()
print(years)

['1873-1874', '1892', '1893', '1894', '1901', '1918', '1921', '1928', '1948', '1956']


<br>

## Aquiring data

In [3]:
# Create an empty list for the final dataframe 
df = []

In [4]:
# Set up the progress bar
progress_bar = tqdm(total=len(years), file=sys.stderr)

for year in years:
    
    # Update the progress bar
    progress_bar.set_description(f"Processing year {year}")
    
    # This is the directory that will contain the OCRed output:
    dir_OCR = ocred_path + '/' + str(year)
    
    if 'acts_path' in globals():
        del acts_path
        
    acts_path, actsSep = getActsPaths(dir_OCR)
    
    if acts_path is None:
        sys.exit(f'acts path not found for {year}')

    with open(acts_path, 'r') as f:
        # This variable holds all the OCRed text as a String
        data = f.read()
        
    count = data.count("\n\n")+1
    if count < 100:
        sys.exit(f'Count seems low for {year}. Count: {count}')
    # print("The number of pages OCRed for {year} is: {count}".format(year = year, count = count))
    
    # Training the tokenizer
    sent_tokenizer = nltk.PunktSentenceTokenizer(data)
    sentences = sent_tokenizer.tokenize(data)  # A List of tokens/sentences as seperated by nltk's PunktSentenceTokenizer

    # Create a temporary dataframe
    df_temp = pd.DataFrame()
    df_temp["sentence"] = sentences
    
    # A flag to keep track of the first index for each year
    df_temp['first'] = False
    
    # Strip sentences of trailing and leading whitespaces
    df_temp['sentence'] = df_temp['sentence'].str.strip()
    # Remove "\n\n" from the original dataframe as they will interfere with the analysis
    df_temp['sentence'] = df_temp['sentence'].str.replace("\n\n", "", regex = False)
    
    # print("Length of the initial dataframe:", df.shape[0], "\nThis is the number of tokenized sentences.")
    
    # Remove session headers
    df_temp = removeSessionHeaders(df_temp)
    
    imgs, dir_imgs = getImgs(dir_OCR, year)
    # print("The number of image files for this year is:", len(imgs))
    
    fileType = imgs[0].split(".")[1]
    
    df_words = getWordsFrame(acts_path, actsSep)
    
    # Add an empty 'start_page' and 'end_page' column
    df_temp['start_page'] = pd.NA
    df_temp['end_page'] = pd.NA
    
    df_temp = getStartEndPages(df_temp, df_words)
    
    # Adding year
    df_temp.insert(0, 'year', year)
    
    # Get the images path
    df_temp = getImgsPath(df_temp, fileType, dir_imgs)
    
    df_temp.at[0, 'first'] = True
    
    # Append this year's dataframe to the final dataframe
    df.append(df_temp)
    
    # Update the progress bar
    progress_bar.update(1)

# Close the progress bar
progress_bar.set_description(f"Processed the list")
progress_bar.close()

# Convert the list to a dataframe
df = pd.concat(df, ignore_index=True)

Processed the list: 100%|██████████| 10/10 [00:13<00:00,  1.32s/it]      


In [5]:
print(f"Length of the dataframe: {df.shape[0]}")
display(df)

Length of the dataframe: 39539


,year,sentence,first,start_page,end_page,path
0,1873-1874,"AN ACT to Repeat Section Four (4) or an AcT ENTITLED “An Act To RELIEVE THE STATE OF SOUTH CAROLINA OF ALL LIABILITY FOR ITS GUARANTY OF THE Bonps OF THE BLUE Ripce RaiLRoap Company, BY PROVIDING FOR THE SECURING AND DESTRUCTION OF THE SAME,” APPROVED Marcu 2, 1872; AND TO REPEAL sO MUCH OF SECTION 72, CHaprer XII, TitLE III, or THE GENERAL STATUTES, AS AUTHORIZES THE STATE AUDITOR TO GIVE Notice ANNUALLY TO Eacu County AvupI- TOR OF THE Rates PER CENTUM TO BE LEVIED FOR VARIOUS SratE PuRpPosEs.",True,00000031,00000031,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg
1,1873-1874,"| Section 1. Be it enacted by the Senate and House of Represen- tatives of the State of South Carolina, now met and sitting in Gen- eral Assembly, and by the authority of the same, That Section four (4) of an Act entitled “An Act to relieve the State of South Caro- lina of all liability for its guaranty of the bonds of the Blue Ridge Railroad Company, by providing for the securing and destruction of the same, approved March 2, 1872, providing for an annual tax of three mills on the dollar, for the redemption of the revenue bond scrip, be, and the same is hereby, repealed.",False,00000031,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg
2,1873-1874,"Sec. 2. That so much of Section seventy-two (72), Chapter XII, Title III, as directs that the State Auditor shall, on or before November fifteenth, annually, give notice to each County Auditor of the rates per centum authorized by law to be levied for various State pur- poses, be, and the same is hereby, repealed, and the Comptroller General is hereby forbidden to levy any tax for any purpose what- ever, unless expressly hereafter authorized so to do by statute.",False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg
3,1873-1874,"Approved October 22, 1873. AN ACT to Revive, RENEw AND AMEND AN ACT ENTITLED “An Acr to Incorporate THE Home InsurANcE Com- PANY, OF CHARLESTON.” Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the Act of the General As- sembly of the said State, entitled “ An Act to incorporate the Home Insurance Company, of Charleston,” be, and the same is hereby, revived, renewed and extended, from the passage hereof, with the following alterations and amendments, viz: Section 5, strike out the words “one year” and insert in lieu thereof the words “ two years.” Section 8, after the word “ respondentia,” insert the words “or other securities.” Section 14, strike out the words “one year” and insert in lieu thereof “ two years.” Approved, November 7, 1873. —_—_— AN ACT To IncorPorRaTE THE REFORM Apotto Soclery, OF CHARLESTON, SouTH CAROLINA.",False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg
4,1873-1874,"Section 1. Be it enacted by the Senate and House of Represen- tatives of the State of South Carolina, now met and sitting in Gen- eral Assembly, and by the authority of the same, That Nathaniel T. Spencer, Robert Howard, Jr., Alfred F. Castion, Edward P. Wall, Jr., Theodore L. Castion, Peter Grant, Eugene M. Castion, Claudius Elliott, their associates and successors, are hereby made and created a body politic and corporate, under the name and style of ‘The Reform Apollo Society, of Charleston, South Carolina.” Sec. 2.",False,00000032,00000033,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg
...,...,...,...,...,...,...
39534,1956,"The Commissioner of Agriculture

<br>

## Pre-Cleaning

In [6]:
# New dataframe so that the results of the matching can be compared
df_cleaned = df.copy()

# A new dictionary to keep track of the number of errors
errorsDict = {}

### Correcting Some Important Words

In [7]:
target_words = ['section']  # Add more target words
print("Correcting the following word spellings in the dataframe:", target_words)

Correcting the following word spellings in the dataframe: ['section']


In [8]:
num_cores = multiprocessing.cpu_count()
chunks = array_split(df_cleaned, num_cores)
print(f'Using {num_cores} cores.')

with multiprocessing.Pool(num_cores) as pool:
    threshold = 1.5  # Adjust the threshold as needed
    
    params = [(chunk, target_words, threshold) for chunk in chunks]    
    processed_chunks = pool.starmap(correct_chunk, params)

df_cleaned = pd.concat(processed_chunks, ignore_index=True)

Using 8 cores.


In [9]:
df_cleaned.drop(['sentence'], axis = 1, inplace=True)
df_cleaned.rename({'corrected_sentence':'sentence'}, axis=1, inplace=True)

In [10]:
df_cleaned.head()

,year,first,start_page,end_page,path,sentence,flag,org_words
0,1873-1874,True,00000031,00000031,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"AN ACT to Repeat Section Four (4) or an AcT ENTITLED “An Act To RELIEVE THE STATE OF SOUTH CAROLINA OF ALL LIABILITY FOR ITS GUARANTY OF THE Bonps OF THE BLUE Ripce RaiLRoap Company, BY PROVIDING FOR THE SECURING AND DESTRUCTION OF THE SAME,” APPROVED Marcu 2, 1872; AND TO REPEAL sO MUCH OF SECTION 72, CHaprer XII, TitLE III, or THE GENERAL STATUTES, AS AUTHORIZES THE STATE AUDITOR TO GIVE Notice ANNUALLY TO Eacu County AvupI- TOR OF THE Rates PER CENTUM TO BE LEVIED FOR VARIOUS SratE PuRpPosEs.",False,
1,1873-1874,False,00000031,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"| Section 1. Be it enacted by the Senate and House of Represen- tatives of the State of South Carolina, now met and sitting in Gen- eral Assembly, and by the authority of the same, That Section four (4) of an Act entitled “An Act to relieve the State of South Caro- lina of all liability for its guaranty of the bonds of the Blue Ridge Railroad Company, by providing for the securing and destruction of the same, approved March 2, 1872, providing for an annual tax of three mills on the dollar, for the redemption of the revenue bond scrip, be, and the same is hereby, repealed.",False,
2,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"Sec. 2. That so much of Section seventy-two (72), Chapter XII, Title III, as directs that the State Auditor shall, on or before November fifteenth, annually, give notice to each County Auditor of the rates per centum authorized by law to be levied for various State pur- poses, be, and the same is hereby, repealed, and the Comptroller General is hereby forbidden to levy any tax for any purpose what- ever, unless expressly hereafter authorized so to do by statute.",False,
3,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"Approved October 22, 1873. AN ACT to Revive, RENEw AND AMEND AN ACT ENTITLED “An Acr to Incorporate THE Home InsurANcE Com- PANY, OF CHARLESTON.” Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the Act of the General As- sembly of the said State, entitled “ An Act to incorporate the Home Insurance Company, of Charleston,” be, and the same is hereby, revived, renewed and extended, from the passage hereof, with the following alterations and amendments, viz: Section 5, strike out the words “one year” and insert in lieu thereof the words “ two years.” Section 8, after the word “ respondentia,” insert the words “or other securities.” Section 14, strike out the words “one year” and insert in lieu thereof “ two years.” Approved, November 7, 1873. —_—_— AN ACT To IncorPorRaTE THE REFORM Apotto Soclery, OF CHARLESTON, SouTH CAROLINA.",False,
4,1873-1874,False,00000032,00000033,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"Section 1. Be it enacted by the Senate and House of Represen- tatives of the State of South Carolina, now met and sitting in Gen- eral Assembly, and by the authority of the same, That Nathaniel T. Spencer, Robert Howard, Jr., Alfred F. Castion, Edward P. Wall, Jr., Theodore L. Castion, Peter Grant, Eugene M. Castion, Claudius Elliott, their associates and successors, are hereby made and created a body politic and corporate, under the name and style of ‘The Reform Apollo Society, of Charleston, South Carolina.” Sec. 2.",False,


In [11]:
df_cleaned['org_words'].value_counts().head()

            38369
“section      458
sections      333
seotion       146
srction        43
Name: org_words, dtype: int64

### Splitting Sentences Based On "Approved ..." Phrases
Some "Approved..." phrases appear at the end of an Act. Sometimes a phrase, such as a new Act, might also by added on to the end of this phrase.
The phrases after the incorrect "Approved" phrases should be split into a new sentence.

In [12]:
approved_rgx_strings = [r'(approved the [0Oo1Iil!2Z5S6G\d]{1,2}(t|h|s|t|n|d|r|d){0,2} day of [a-z]+(,| |.){1,2}((a\. d\.(. |.| |){1,2}){0,1}.{4}(. |.| |)){0,1})', 
                        r'(approved( |,|){0,2}[a-z]+ [0Oo1Iil!2Z5S6G\d]{1,2}(t|h|s|t|n|d|r|d){0,2}, (a\. d\.(. |.| |){1,2}){0,1}.{4}(. |.| |))', 
                        r'(approved the [a-z]+-[a-z]+ day of [a-z]+, (a\. d\.(. |.| |){1,2}){0,1}.{4}(. |.| |))', 
                        r'(approved: [a-z| |.]+, [a-z]+.)', 
                        r'(approved( |,|){0,2}[a-z]+(.| |,){1,2}[0Oo1Iil!2Z5S6G\d]{1,2}(.| |,){0,1})']

In [13]:
approved_rgx = []
for string in approved_rgx_strings:
    approved_rgx.append( re.compile(string, re.IGNORECASE) )

In [14]:
approved_rgx

[re.compile(r'(approved the [0Oo1Iil!2Z5S6G\d]{1,2}(t|h|s|t|n|d|r|d){0,2} day of [a-z]+(,| |.){1,2}((a\. d\.(. |.| |){1,2}){0,1}.{4}(. |.| |)){0,1})',
            re.IGNORECASE|re.UNICODE),
 re.compile(r'(approved( |,|){0,2}[a-z]+ [0Oo1Iil!2Z5S6G\d]{1,2}(t|h|s|t|n|d|r|d){0,2}, (a\. d\.(. |.| |){1,2}){0,1}.{4}(. |.| |))',
            re.IGNORECASE|re.UNICODE),
 re.compile(r'(approved the [a-z]+-[a-z]+ day of [a-z]+, (a\. d\.(. |.| |){1,2}){0,1}.{4}(. |.| |))',
            re.IGNORECASE|re.UNICODE),
 re.compile(r'(approved: [a-z| |.]+, [a-z]+.)', re.IGNORECASE|re.UNICODE),
 re.compile(r'(approved( |,|){0,2}[a-z]+(.| |,){1,2}[0Oo1Iil!2Z5S6G\d]{1,2}(.| |,){0,1})',
            re.IGNORECASE|re.UNICODE)]

In [15]:
df_new = df_cleaned.copy()

for i, rgx in enumerate(approved_rgx_strings):
    print(f'Working on pattern {i+1} using {cpu_count()} cores.')
    
    # Split the DataFrame using multiprocessing
    new_rows = []

    # Create a multiprocessing pool with the number of desired processes
    with multiprocessing.Pool() as pool:
        args_list = [(row, rgx, list(df_new.columns)) for _, row in df_new.iterrows()]

        # Use starmap to pass the tuples as separate arguments to process_row
        results = pool.starmap(process_row, args_list)

    # Flatten the list of lists into a single list of split rows
    new_rows = [item for sublist in results for item in sublist]

    # Create a new DataFrame with the split rows
    df_new = pd.DataFrame(new_rows)

Working on pattern 1 using 8 cores.
Working on pattern 2 using 8 cores.
Working on pattern 3 using 8 cores.
Working on pattern 4 using 8 cores.
Working on pattern 5 using 8 cores.


In [16]:
df_new.head()

,year,first,start_page,end_page,path,sentence,flag,org_words
0,1873-1874,True,00000031,00000031,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"AN ACT to Repeat Section Four (4) or an AcT ENTITLED “An Act To RELIEVE THE STATE OF SOUTH CAROLINA OF ALL LIABILITY FOR ITS GUARANTY OF THE Bonps OF THE BLUE Ripce RaiLRoap Company, BY PROVIDING FOR THE SECURING AND DESTRUCTION OF THE SAME,” APPROVED Marcu 2, 1872; AND TO REPEAL sO MUCH OF SECTION 72, CHaprer XII, TitLE III, or THE GENERAL STATUTES, AS AUTHORIZES THE STATE AUDITOR TO GIVE Notice ANNUALLY TO Eacu County AvupI- TOR OF THE Rates PER CENTUM TO BE LEVIED FOR VARIOUS SratE PuRpPosEs.",False,
1,1873-1874,False,00000031,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"| Section 1. Be it enacted by the Senate and House of Represen- tatives of the State of South Carolina, now met and sitting in Gen- eral Assembly, and by the authority of the same, That Section four (4) of an Act entitled “An Act to relieve the State of South Caro- lina of all liability for its guaranty of the bonds of the Blue Ridge Railroad Company, by providing for the securing and destruction of the same, approved March 2, 1872, providing for an annual tax of three mills on the dollar, for the redemption of the revenue bond scrip, be, and the same is hereby, repealed.",False,
2,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"Sec. 2. That so much of Section seventy-two (72), Chapter XII, Title III, as directs that the State Auditor shall, on or before November fifteenth, annually, give notice to each County Auditor of the rates per centum authorized by law to be levied for various State pur- poses, be, and the same is hereby, repealed, and the Comptroller General is hereby forbidden to levy any tax for any purpose what- ever, unless expressly hereafter authorized so to do by statute.",False,
3,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"Approved October 22, 1873. AN ACT to Revive, RENEw AND AMEND AN ACT ENTITLED “An Acr to Incorporate THE Home InsurANcE Com- PANY, OF CHARLESTON.” Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the Act of the General As- sembly of the said State, entitled “ An Act to incorporate the Home Insurance Company, of Charleston,” be, and the same is hereby, revived, renewed and extended, from the passage hereof, with the following alterations and amendments, viz: Section 5, strike out the words “one year” and insert in lieu thereof the words “ two years.” Section 8, after the word “ respondentia,” insert the words “or other securities.” Section 14, strike out the words “one year” and insert in lieu thereof “ two years.”",False,
4,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"—_—_— AN ACT To IncorPorRaTE THE REFORM Apotto Soclery, OF CHARLESTON, SouTH CAROLINA.",False,


In [17]:
if df_new.shape[0] > df_cleaned.shape[0]:
    errorsDict['Split Sentences'] = df_new.shape[0] - df_cleaned.shape[0]
    df_cleaned = df_new
    display(errorsDict)

{'Split Sentences': 190}

### Relocating Incorrect "Approved ..." Phrases
Since "Approved..." phrases appear in different formats throughout the years, the code uses multiple Regex strings to match the first occurence.
<br>By testing on the entire corpus, these 5 strings match the formats in all years. However, there will be some outliers (errors) that can not be captured by these strings.

In [18]:
# Add the caret in the regex strings...
for i, string in enumerate(approved_rgx_strings):
    approved_rgx_strings[i] = '^' + string
approved_rgx_strings

['^(approved the [0Oo1Iil!2Z5S6G\\d]{1,2}(t|h|s|t|n|d|r|d){0,2} day of [a-z]+(,| |.){1,2}((a\\. d\\.(. |.| |){1,2}){0,1}.{4}(. |.| |)){0,1})',
 '^(approved( |,|){0,2}[a-z]+ [0Oo1Iil!2Z5S6G\\d]{1,2}(t|h|s|t|n|d|r|d){0,2}, (a\\. d\\.(. |.| |){1,2}){0,1}.{4}(. |.| |))',
 '^(approved the [a-z]+-[a-z]+ day of [a-z]+, (a\\. d\\.(. |.| |){1,2}){0,1}.{4}(. |.| |))',
 '^(approved: [a-z| |.]+, [a-z]+.)',
 '^(approved( |,|){0,2}[a-z]+(.| |,){1,2}[0Oo1Iil!2Z5S6G\\d]{1,2}(.| |,){0,1})']

In [19]:
approved_rgx = []
for string in approved_rgx_strings:
    approved_rgx.append( re.compile(string, re.IGNORECASE) )

In [20]:
approved_rgx

[re.compile(r'^(approved the [0Oo1Iil!2Z5S6G\d]{1,2}(t|h|s|t|n|d|r|d){0,2} day of [a-z]+(,| |.){1,2}((a\. d\.(. |.| |){1,2}){0,1}.{4}(. |.| |)){0,1})',
            re.IGNORECASE|re.UNICODE),
 re.compile(r'^(approved( |,|){0,2}[a-z]+ [0Oo1Iil!2Z5S6G\d]{1,2}(t|h|s|t|n|d|r|d){0,2}, (a\. d\.(. |.| |){1,2}){0,1}.{4}(. |.| |))',
            re.IGNORECASE|re.UNICODE),
 re.compile(r'^(approved the [a-z]+-[a-z]+ day of [a-z]+, (a\. d\.(. |.| |){1,2}){0,1}.{4}(. |.| |))',
            re.IGNORECASE|re.UNICODE),
 re.compile(r'^(approved: [a-z| |.]+, [a-z]+.)', re.IGNORECASE|re.UNICODE),
 re.compile(r'^(approved( |,|){0,2}[a-z]+(.| |,){1,2}[0Oo1Iil!2Z5S6G\d]{1,2}(.| |,){0,1})',
            re.IGNORECASE|re.UNICODE)]

In [21]:
errorsDict['Approved phrases'] = 0

for rgx_match in approved_rgx:
    
    # Search for matches in the 'sentence' column
    matches = df_cleaned['sentence'].str.extract(rgx_match)[0]
    
    # Remove the matched patterns from sentences
    df_cleaned['sentence'] = df_cleaned['sentence'].str.replace(rgx_match, '', n=-1)
    
    # Add matches to the end of previous sentence
    df_cleaned['sentence'] = df_cleaned['sentence'].str.cat(matches.shift(-1), 
                                                            sep=' ', na_rep='')
    
    errorsDict['Approved phrases'] += matches.count().sum()

In [22]:
errorsDict

{'Split Sentences': 190, 'Approved phrases': 2380}

In [23]:
df_cleaned.head()

,year,first,start_page,end_page,path,sentence,flag,org_words
0,1873-1874,True,00000031,00000031,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"AN ACT to Repeat Section Four (4) or an AcT ENTITLED “An Act To RELIEVE THE STATE OF SOUTH CAROLINA OF ALL LIABILITY FOR ITS GUARANTY OF THE Bonps OF THE BLUE Ripce RaiLRoap Company, BY PROVIDING FOR THE SECURING AND DESTRUCTION OF THE SAME,” APPROVED Marcu 2, 1872; AND TO REPEAL sO MUCH OF SECTION 72, CHaprer XII, TitLE III, or THE GENERAL STATUTES, AS AUTHORIZES THE STATE AUDITOR TO GIVE Notice ANNUALLY TO Eacu County AvupI- TOR OF THE Rates PER CENTUM TO BE LEVIED FOR VARIOUS SratE PuRpPosEs.",False,
1,1873-1874,False,00000031,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"| Section 1. Be it enacted by the Senate and House of Represen- tatives of the State of South Carolina, now met and sitting in Gen- eral Assembly, and by the authority of the same, That Section four (4) of an Act entitled “An Act to relieve the State of South Caro- lina of all liability for its guaranty of the bonds of the Blue Ridge Railroad Company, by providing for the securing and destruction of the same, approved March 2, 1872, providing for an annual tax of three mills on the dollar, for the redemption of the revenue bond scrip, be, and the same is hereby, repealed.",False,
2,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"Sec. 2. That so much of Section seventy-two (72), Chapter XII, Title III, as directs that the State Auditor shall, on or before November fifteenth, annually, give notice to each County Auditor of the rates per centum authorized by law to be levied for various State pur- poses, be, and the same is hereby, repealed, and the Comptroller General is hereby forbidden to levy any tax for any purpose what- ever, unless expressly hereafter authorized so to do by statute. Approved October 22, 1873.",False,
3,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT to Revive, RENEw AND AMEND AN ACT ENTITLED “An Acr to Incorporate THE Home InsurANcE Com- PANY, OF CHARLESTON.” Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the Act of the General As- sembly of the said State, entitled “ An Act to incorporate the Home Insurance Company, of Charleston,” be, and the same is hereby, revived, renewed and extended, from the passage hereof, with the following alterations and amendments, viz: Section 5, strike out the words “one year” and insert in lieu thereof the words “ two years.” Section 8, after the word “ respondentia,” insert the words “or other securities.” Section 14, strike out the words “one year” and insert in lieu thereof “ two years.”",False,
4,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"—_—_— AN ACT To IncorPorRaTE THE REFORM Apotto Soclery, OF CHARLESTON, SouTH CAROLINA.",False,


### Removing End-Of-Line Hyphenation

In [24]:
errorsDict['EOL hyphenation'] = df_cleaned['sentence'].str.count(pat = r'(—|_|-)( )*').sum()
df_cleaned['sentence'] = df_cleaned['sentence'].str.replace(pat = r'(—|_|-)( )*',
                                                            repl = "",
                                                            regex = True)

In [25]:
errorsDict

{'Split Sentences': 190, 'Approved phrases': 2380, 'EOL hyphenation': 42906}

In [26]:
df_cleaned.head()

,year,first,start_page,end_page,path,sentence,flag,org_words
0,1873-1874,True,00000031,00000031,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"AN ACT to Repeat Section Four (4) or an AcT ENTITLED “An Act To RELIEVE THE STATE OF SOUTH CAROLINA OF ALL LIABILITY FOR ITS GUARANTY OF THE Bonps OF THE BLUE Ripce RaiLRoap Company, BY PROVIDING FOR THE SECURING AND DESTRUCTION OF THE SAME,” APPROVED Marcu 2, 1872; AND TO REPEAL sO MUCH OF SECTION 72, CHaprer XII, TitLE III, or THE GENERAL STATUTES, AS AUTHORIZES THE STATE AUDITOR TO GIVE Notice ANNUALLY TO Eacu County AvupITOR OF THE Rates PER CENTUM TO BE LEVIED FOR VARIOUS SratE PuRpPosEs.",False,
1,1873-1874,False,00000031,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"| Section 1. Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That Section four (4) of an Act entitled “An Act to relieve the State of South Carolina of all liability for its guaranty of the bonds of the Blue Ridge Railroad Company, by providing for the securing and destruction of the same, approved March 2, 1872, providing for an annual tax of three mills on the dollar, for the redemption of the revenue bond scrip, be, and the same is hereby, repealed.",False,
2,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"Sec. 2. That so much of Section seventytwo (72), Chapter XII, Title III, as directs that the State Auditor shall, on or before November fifteenth, annually, give notice to each County Auditor of the rates per centum authorized by law to be levied for various State purposes, be, and the same is hereby, repealed, and the Comptroller General is hereby forbidden to levy any tax for any purpose whatever, unless expressly hereafter authorized so to do by statute. Approved October 22, 1873.",False,
3,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT to Revive, RENEw AND AMEND AN ACT ENTITLED “An Acr to Incorporate THE Home InsurANcE ComPANY, OF CHARLESTON.” Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the Act of the General Assembly of the said State, entitled “ An Act to incorporate the Home Insurance Company, of Charleston,” be, and the same is hereby, revived, renewed and extended, from the passage hereof, with the following alterations and amendments, viz: Section 5, strike out the words “one year” and insert in lieu thereof the words “ two years.” Section 8, after the word “ respondentia,” insert the words “or other securities.” Section 14, strike out the words “one year” and insert in lieu thereof “ two years.”",False,
4,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT To IncorPorRaTE THE REFORM Apotto Soclery, OF CHARLESTON, SouTH CAROLINA.",False,


### Removing Act Seperators

In [27]:
errorsDict['Act seperators'] = df_cleaned['sentence'].str.count(pat = r'^—+(?=\s*[A-Za-z])').sum()
df_cleaned['sentence'] = df_cleaned['sentence'].str.replace(pat = r'^—+(?=\s*[A-Za-z])',
                                                            repl = '',
                                                            regex = True)

In [28]:
errorsDict

{'Split Sentences': 190,
 'Approved phrases': 2380,
 'EOL hyphenation': 42906,
 'Act seperators': 0}

In [29]:
print(f"Length of the dataframe: {df_cleaned.shape[0]}")

Length of the dataframe: 39729


<br>

## Adding Section and Act Labels
Add Section and Act labels for each sentence.

In [30]:
df_updated = df_cleaned.copy()

In [31]:
def getAct(sentence, fallback, first, pattern, group = 1):
    """
    Get Act labels for the given sentence.
    If the match is an empty string (’’) then the last act is updated by 1.
    Else the matched act number is the new act number.

    Note: Joint Resolutions might be present in the OCRed text, 
    but are not needed here. So, to maintain continuity with Act labels, 
    Joint Resolutions are labeled as a new Act.

    Requires a lastAct variable to keep track of the previous act.
    
    Parameters
    ----------
    sentence : str
        The sentence to output the act for.
    fallback : str
        If no act is found, then return this str instead.
    pat : re.Pattern
        The pattern to search for.
    group : int
        The group to extract from the match.

    Returns
    -------
    str
        The Act number for this `sentence`.
    """  

    global lastAct
    res = pattern.search(sentence.lower())
    
    # Since no way to detect any Act numbers with these volumes,
    # we need a way to re-initialize `lastAct` if a new volume starts
    if first:
        lastAct = '0'

    # If a Joint Resolution starts with this sentence...
    if 'joint' in sentence.lower().split()[:3] and 'resolution' in sentence.lower().split()[:4]:
        lastAct = str(int(lastAct) + 1)
        return lastAct
    
    # Else if a new Act starts...
    elif res:
        if res.group(group) == '':  # Act number not given, but act is started
            lastAct = str(int(lastAct) + 1)

        else:  # Act number given
            lastAct = res.group(group)

        return lastAct
    
    return fallback

In [32]:
df_updated['act'] = None

# Compile the regex pattern
pattern = re.compile(r'^([\d]*)([. ]*)(?:an act|act)')

# Initialize lastAct
lastAct = '0'

# Apply the function to the DataFrame
df_updated['act'] = df_updated.apply(lambda x: getAct(x['sentence'], x['act'], x['first'], pattern, 1), axis=1)

In [33]:
df_updated.head()

,year,first,start_page,end_page,path,sentence,flag,org_words,act
0,1873-1874,True,00000031,00000031,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"AN ACT to Repeat Section Four (4) or an AcT ENTITLED “An Act To RELIEVE THE STATE OF SOUTH CAROLINA OF ALL LIABILITY FOR ITS GUARANTY OF THE Bonps OF THE BLUE Ripce RaiLRoap Company, BY PROVIDING FOR THE SECURING AND DESTRUCTION OF THE SAME,” APPROVED Marcu 2, 1872; AND TO REPEAL sO MUCH OF SECTION 72, CHaprer XII, TitLE III, or THE GENERAL STATUTES, AS AUTHORIZES THE STATE AUDITOR TO GIVE Notice ANNUALLY TO Eacu County AvupITOR OF THE Rates PER CENTUM TO BE LEVIED FOR VARIOUS SratE PuRpPosEs.",False,,1
1,1873-1874,False,00000031,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"| Section 1. Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That Section four (4) of an Act entitled “An Act to relieve the State of South Carolina of all liability for its guaranty of the bonds of the Blue Ridge Railroad Company, by providing for the securing and destruction of the same, approved March 2, 1872, providing for an annual tax of three mills on the dollar, for the redemption of the revenue bond scrip, be, and the same is hereby, repealed.",False,,None
2,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"Sec. 2. That so much of Section seventytwo (72), Chapter XII, Title III, as directs that the State Auditor shall, on or before November fifteenth, annually, give notice to each County Auditor of the rates per centum authorized by law to be levied for various State purposes, be, and the same is hereby, repealed, and the Comptroller General is hereby forbidden to levy any tax for any purpose whatever, unless expressly hereafter authorized so to do by statute. Approved October 22, 1873.",False,,None
3,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT to Revive, RENEw AND AMEND AN ACT ENTITLED “An Acr to Incorporate THE Home InsurANcE ComPANY, OF CHARLESTON.” Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the Act of the General Assembly of the said State, entitled “ An Act to incorporate the Home Insurance Company, of Charleston,” be, and the same is hereby, revived, renewed and extended, from the passage hereof, with the following alterations and amendments, viz: Section 5, strike out the words “one year” and insert in lieu thereof the words “ two years.” Section 8, after the word “ respondentia,” insert the words “or other securities.” Section 14, strike out the words “one year” and insert in lieu thereof “ two years.”",False,,2
4,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT To IncorPorRaTE THE REFORM Apotto Soclery, OF CHARLESTON, SouTH CAROLINA.",False,,3


<br>

In [34]:
# Go through sentences and label some occurences of new sections with their numbers
pattern = re.compile(r'^([\d]*)([. ]*)(?:an act|act).*?(?:section)\s+([\w]{1,4})[\s.]*(?:be it)')
df_updated['section_rgx1'] = df_updated['sentence'].str.lower().str.extract(pattern)[2]

In [35]:
df_updated.head()

,year,first,start_page,end_page,path,sentence,flag,org_words,act,section_rgx1
0,1873-1874,True,00000031,00000031,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"AN ACT to Repeat Section Four (4) or an AcT ENTITLED “An Act To RELIEVE THE STATE OF SOUTH CAROLINA OF ALL LIABILITY FOR ITS GUARANTY OF THE Bonps OF THE BLUE Ripce RaiLRoap Company, BY PROVIDING FOR THE SECURING AND DESTRUCTION OF THE SAME,” APPROVED Marcu 2, 1872; AND TO REPEAL sO MUCH OF SECTION 72, CHaprer XII, TitLE III, or THE GENERAL STATUTES, AS AUTHORIZES THE STATE AUDITOR TO GIVE Notice ANNUALLY TO Eacu County AvupITOR OF THE Rates PER CENTUM TO BE LEVIED FOR VARIOUS SratE PuRpPosEs.",False,,1,NaN
1,1873-1874,False,00000031,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"| Section 1. Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That Section four (4) of an Act entitled “An Act to relieve the State of South Carolina of all liability for its guaranty of the bonds of the Blue Ridge Railroad Company, by providing for the securing and destruction of the same, approved March 2, 1872, providing for an annual tax of three mills on the dollar, for the redemption of the revenue bond scrip, be, and the same is hereby, repealed.",False,,None,NaN
2,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"Sec. 2. That so much of Section seventytwo (72), Chapter XII, Title III, as directs that the State Auditor shall, on or before November fifteenth, annually, give notice to each County Auditor of the rates per centum authorized by law to be levied for various State purposes, be, and the same is hereby, repealed, and the Comptroller General is hereby forbidden to levy any tax for any purpose whatever, unless expressly hereafter authorized so to do by statute. Approved October 22, 1873.",False,,None,NaN
3,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT to Revive, RENEw AND AMEND AN ACT ENTITLED “An Acr to Incorporate THE Home InsurANcE ComPANY, OF CHARLESTON.” Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the Act of the General Assembly of the said State, entitled “ An Act to incorporate the Home Insurance Company, of Charleston,” be, and the same is hereby, revived, renewed and extended, from the passage hereof, with the following alterations and amendments, viz: Section 5, strike out the words “one year” and insert in lieu thereof the words “ two years.” Section 8, after the word “ respondentia,” insert the words “or other securities.” Section 14, strike out the words “one year” and insert in lieu thereof “ two years.”",False,,2,NaN
4,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT To IncorPorRaTE THE REFORM Apotto Soclery, OF CHARLESTON, SouTH CAROLINA.",False,,3,NaN


<br>

In [36]:
# Go through sentences and label some occurences of new sections with their numbers
pattern = re.compile(r'^(S|s|E|e|r|C|c){1,}(T|t|I|i|O|o|N|n)*(\.|,|:|;| ){0,2}([\d]{1,3}[\w]?)(. |.| |){1,3}')
df_updated['section_rgx3'] = df_updated['sentence'].str.lower().str.extract(pattern)[3]

In [37]:
df_updated.head()

,year,first,start_page,end_page,path,sentence,flag,org_words,act,section_rgx1,section_rgx3
0,1873-1874,True,00000031,00000031,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"AN ACT to Repeat Section Four (4) or an AcT ENTITLED “An Act To RELIEVE THE STATE OF SOUTH CAROLINA OF ALL LIABILITY FOR ITS GUARANTY OF THE Bonps OF THE BLUE Ripce RaiLRoap Company, BY PROVIDING FOR THE SECURING AND DESTRUCTION OF THE SAME,” APPROVED Marcu 2, 1872; AND TO REPEAL sO MUCH OF SECTION 72, CHaprer XII, TitLE III, or THE GENERAL STATUTES, AS AUTHORIZES THE STATE AUDITOR TO GIVE Notice ANNUALLY TO Eacu County AvupITOR OF THE Rates PER CENTUM TO BE LEVIED FOR VARIOUS SratE PuRpPosEs.",False,,1,NaN,NaN
1,1873-1874,False,00000031,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"| Section 1. Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That Section four (4) of an Act entitled “An Act to relieve the State of South Carolina of all liability for its guaranty of the bonds of the Blue Ridge Railroad Company, by providing for the securing and destruction of the same, approved March 2, 1872, providing for an annual tax of three mills on the dollar, for the redemption of the revenue bond scrip, be, and the same is hereby, repealed.",False,,None,NaN,NaN
2,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"Sec. 2. That so much of Section seventytwo (72), Chapter XII, Title III, as directs that the State Auditor shall, on or before November fifteenth, annually, give notice to each County Auditor of the rates per centum authorized by law to be levied for various State purposes, be, and the same is hereby, repealed, and the Comptroller General is hereby forbidden to levy any tax for any purpose whatever, unless expressly hereafter authorized so to do by statute. Approved October 22, 1873.",False,,None,NaN,2
3,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT to Revive, RENEw AND AMEND AN ACT ENTITLED “An Acr to Incorporate THE Home InsurANcE ComPANY, OF CHARLESTON.” Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the Act of the General Assembly of the said State, entitled “ An Act to incorporate the Home Insurance Company, of Charleston,” be, and the same is hereby, revived, renewed and extended, from the passage hereof, with the following alterations and amendments, viz: Section 5, strike out the words “one year” and insert in lieu thereof the words “ two years.” Section 8, after the word “ respondentia,” insert the words “or other securities.” Section 14, strike out the words “one year” and insert in lieu thereof “ two years.”",False,,2,NaN,NaN
4,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT To IncorPorRaTE THE REFORM Apotto Soclery, OF CHARLESTON, SouTH CAROLINA.",False,,3,NaN,NaN


<br>

In [38]:
# Go through sentences and label some occurences of new sections with their numbers
pattern = re.compile(r'(s|e|r|c){1,}(t|i|o|n)*(\.|,|:|;| ){0,2}([\d]{1,3}[\w]?)(. |.| |){1,3}$')

# Search for matches in the 'sentence' column
matches = df_updated['sentence'].str.lower().str.extract(pattern)[3]
# Add match to the next row
matches = matches.shift(1)

df_updated['section_rgx4'] = matches

# Replace the first 'None' value to NaN
df_updated.at[0, 'section_rgx4'] = nan

In [39]:
df_updated.head()

,year,first,start_page,end_page,path,sentence,flag,org_words,act,section_rgx1,section_rgx3,section_rgx4
0,1873-1874,True,00000031,00000031,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"AN ACT to Repeat Section Four (4) or an AcT ENTITLED “An Act To RELIEVE THE STATE OF SOUTH CAROLINA OF ALL LIABILITY FOR ITS GUARANTY OF THE Bonps OF THE BLUE Ripce RaiLRoap Company, BY PROVIDING FOR THE SECURING AND DESTRUCTION OF THE SAME,” APPROVED Marcu 2, 1872; AND TO REPEAL sO MUCH OF SECTION 72, CHaprer XII, TitLE III, or THE GENERAL STATUTES, AS AUTHORIZES THE STATE AUDITOR TO GIVE Notice ANNUALLY TO Eacu County AvupITOR OF THE Rates PER CENTUM TO BE LEVIED FOR VARIOUS SratE PuRpPosEs.",False,,1,NaN,NaN,NaN
1,1873-1874,False,00000031,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"| Section 1. Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That Section four (4) of an Act entitled “An Act to relieve the State of South Carolina of all liability for its guaranty of the bonds of the Blue Ridge Railroad Company, by providing for the securing and destruction of the same, approved March 2, 1872, providing for an annual tax of three mills on the dollar, for the redemption of the revenue bond scrip, be, and the same is hereby, repealed.",False,,None,NaN,NaN,NaN
2,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"Sec. 2. That so much of Section seventytwo (72), Chapter XII, Title III, as directs that the State Auditor shall, on or before November fifteenth, annually, give notice to each County Auditor of the rates per centum authorized by law to be levied for various State purposes, be, and the same is hereby, repealed, and the Comptroller General is hereby forbidden to levy any tax for any purpose whatever, unless expressly hereafter authorized so to do by statute. Approved October 22, 1873.",False,,None,NaN,2,NaN
3,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT to Revive, RENEw AND AMEND AN ACT ENTITLED “An Acr to Incorporate THE Home InsurANcE ComPANY, OF CHARLESTON.” Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the Act of the General Assembly of the said State, entitled “ An Act to incorporate the Home Insurance Company, of Charleston,” be, and the same is hereby, revived, renewed and extended, from the passage hereof, with the following alterations and amendments, viz: Section 5, strike out the words “one year” and insert in lieu thereof the words “ two years.” Section 8, after the word “ respondentia,” insert the words “or other securities.” Section 14, strike out the words “one year” and insert in lieu thereof “ two years.”",False,,2,NaN,NaN,NaN
4,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT To IncorPorRaTE THE REFORM Apotto Soclery, OF CHARLESTON, SouTH CAROLINA.",False,,3,NaN,NaN,NaN


<br>

In [40]:
# Go through sentences and label some occurences of new sections with their numbers
pattern = re.compile(r'§(\.|,|:|;| ){0,2}([\d]{1,3}[\w]?)')
df_updated['section_rgx5'] = df_updated['sentence'].str.lower().str.extract(pattern)[1]

In [41]:
df_updated.head()

,year,first,start_page,end_page,path,sentence,flag,org_words,act,section_rgx1,section_rgx3,section_rgx4,section_rgx5
0,1873-1874,True,00000031,00000031,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"AN ACT to Repeat Section Four (4) or an AcT ENTITLED “An Act To RELIEVE THE STATE OF SOUTH CAROLINA OF ALL LIABILITY FOR ITS GUARANTY OF THE Bonps OF THE BLUE Ripce RaiLRoap Company, BY PROVIDING FOR THE SECURING AND DESTRUCTION OF THE SAME,” APPROVED Marcu 2, 1872; AND TO REPEAL sO MUCH OF SECTION 72, CHaprer XII, TitLE III, or THE GENERAL STATUTES, AS AUTHORIZES THE STATE AUDITOR TO GIVE Notice ANNUALLY TO Eacu County AvupITOR OF THE Rates PER CENTUM TO BE LEVIED FOR VARIOUS SratE PuRpPosEs.",False,,1,NaN,NaN,NaN,NaN
1,1873-1874,False,00000031,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"| Section 1. Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That Section four (4) of an Act entitled “An Act to relieve the State of South Carolina of all liability for its guaranty of the bonds of the Blue Ridge Railroad Company, by providing for the securing and destruction of the same, approved March 2, 1872, providing for an annual tax of three mills on the dollar, for the redemption of the revenue bond scrip, be, and the same is hereby, repealed.",False,,None,NaN,NaN,NaN,NaN
2,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"Sec. 2. That so much of Section seventytwo (72), Chapter XII, Title III, as directs that the State Auditor shall, on or before November fifteenth, annually, give notice to each County Auditor of the rates per centum authorized by law to be levied for various State purposes, be, and the same is hereby, repealed, and the Comptroller General is hereby forbidden to levy any tax for any purpose whatever, unless expressly hereafter authorized so to do by statute. Approved October 22, 1873.",False,,None,NaN,2,NaN,NaN
3,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT to Revive, RENEw AND AMEND AN ACT ENTITLED “An Acr to Incorporate THE Home InsurANcE ComPANY, OF CHARLESTON.” Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the Act of the General Assembly of the said State, entitled “ An Act to incorporate the Home Insurance Company, of Charleston,” be, and the same is hereby, revived, renewed and extended, from the passage hereof, with the following alterations and amendments, viz: Section 5, strike out the words “one year” and insert in lieu thereof the words “ two years.” Section 8, after the word “ respondentia,” insert the words “or other securities.” Section 14, strike out the words “one year” and insert in lieu thereof “ two years.”",False,,2,NaN,NaN,NaN,NaN
4,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT To IncorPorRaTE THE REFORM Apotto Soclery, OF CHARLESTON, SouTH CAROLINA.",False,,3,NaN,NaN,NaN,NaN


<br>

In [42]:
# Go through sentences and label some occurences of new sections with their numbers
pattern = re.compile(r'^([0Oo1Iil!2Z5S6G\d]{1,3})(. |.| |)')
df_updated['section_rgx6'] = df_updated['sentence'].str.lower().str.extract(pattern)[0]

<br>

In [43]:
# Combine all section columns
df_updated['section_comb'] = df_updated['section_rgx1'].fillna(df_updated['section_rgx3']).fillna(df_updated['section_rgx4']).fillna(df_updated['section_rgx5']).fillna(df_updated['section_rgx6'])

In [44]:
df_updated.head()

,year,first,start_page,end_page,path,sentence,flag,org_words,act,section_rgx1,section_rgx3,section_rgx4,section_rgx5,section_rgx6,section_comb
0,1873-1874,True,00000031,00000031,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"AN ACT to Repeat Section Four (4) or an AcT ENTITLED “An Act To RELIEVE THE STATE OF SOUTH CAROLINA OF ALL LIABILITY FOR ITS GUARANTY OF THE Bonps OF THE BLUE Ripce RaiLRoap Company, BY PROVIDING FOR THE SECURING AND DESTRUCTION OF THE SAME,” APPROVED Marcu 2, 1872; AND TO REPEAL sO MUCH OF SECTION 72, CHaprer XII, TitLE III, or THE GENERAL STATUTES, AS AUTHORIZES THE STATE AUDITOR TO GIVE Notice ANNUALLY TO Eacu County AvupITOR OF THE Rates PER CENTUM TO BE LEVIED FOR VARIOUS SratE PuRpPosEs.",False,,1,NaN,NaN,NaN,NaN,NaN,NaN
1,1873-1874,False,00000031,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"| Section 1. Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That Section four (4) of an Act entitled “An Act to relieve the State of South Carolina of all liability for its guaranty of the bonds of the Blue Ridge Railroad Company, by providing for the securing and destruction of the same, approved March 2, 1872, providing for an annual tax of three mills on the dollar, for the redemption of the revenue bond scrip, be, and the same is hereby, repealed.",False,,None,NaN,NaN,NaN,NaN,NaN,NaN
2,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"Sec. 2. That so much of Section seventytwo (72), Chapter XII, Title III, as directs that the State Auditor shall, on or before November fifteenth, annually, give notice to each County Auditor of the rates per centum authorized by law to be levied for various State purposes, be, and the same is hereby, repealed, and the Comptroller General is hereby forbidden to levy any tax for any purpose whatever, unless expressly hereafter authorized so to do by statute. Approved October 22, 1873.",False,,None,NaN,2,NaN,NaN,NaN,2
3,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT to Revive, RENEw AND AMEND AN ACT ENTITLED “An Acr to Incorporate THE Home InsurANcE ComPANY, OF CHARLESTON.” Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the Act of the General Assembly of the said State, entitled “ An Act to incorporate the Home Insurance Company, of Charleston,” be, and the same is hereby, revived, renewed and extended, from the passage hereof, with the following alterations and amendments, viz: Section 5, strike out the words “one year” and insert in lieu thereof the words “ two years.” Section 8, after the word “ respondentia,” insert the words “or other securities.” Section 14, strike out the words “one year” and insert in lieu thereof “ two years.”",False,,2,NaN,NaN,NaN,NaN,NaN,NaN
4,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT To IncorPorRaTE THE REFORM Apotto Soclery, OF CHARLESTON, SouTH CAROLINA.",False,,3,NaN,NaN,NaN,NaN,NaN,NaN


<br>

In [45]:
# Initialize lastAct
lastAct = '0'

# Go through sentences and label all occurences of new acts with their section numbers
def labelSections(row):
    """
    Label every sentence that has a new act number (something different from the last row's) 
    with 0 if a section does not already exists there.
    
    Requires lastAct variable to be initialized outside of this function.
    
    Parameters
    ----------
    row : pandas.Dataframe row
        The row to perform operation on

    Returns
    -------
    pandas.Dataframe row
        The modified row.
    """
    
    global lastAct
    
    # If at the first index or if the previous act num is not equal to this act
    if row['first'] == True or lastAct != row['act']:
        
        # If this section value is None, label with 0
        if pd.isnull(row['section_comb']):
            row['section_comb'] = 0
            
    # Update lastAct...
    lastAct = row['act']
     
    return row


# Apply the function to the DataFrame
df_updated = df_updated.apply(labelSections, axis=1)

In [46]:
df_updated.head()

,year,first,start_page,end_page,path,sentence,flag,org_words,act,section_rgx1,section_rgx3,section_rgx4,section_rgx5,section_rgx6,section_comb
0,1873-1874,True,00000031,00000031,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"AN ACT to Repeat Section Four (4) or an AcT ENTITLED “An Act To RELIEVE THE STATE OF SOUTH CAROLINA OF ALL LIABILITY FOR ITS GUARANTY OF THE Bonps OF THE BLUE Ripce RaiLRoap Company, BY PROVIDING FOR THE SECURING AND DESTRUCTION OF THE SAME,” APPROVED Marcu 2, 1872; AND TO REPEAL sO MUCH OF SECTION 72, CHaprer XII, TitLE III, or THE GENERAL STATUTES, AS AUTHORIZES THE STATE AUDITOR TO GIVE Notice ANNUALLY TO Eacu County AvupITOR OF THE Rates PER CENTUM TO BE LEVIED FOR VARIOUS SratE PuRpPosEs.",False,,1,NaN,NaN,NaN,NaN,NaN,0
1,1873-1874,False,00000031,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"| Section 1. Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That Section four (4) of an Act entitled “An Act to relieve the State of South Carolina of all liability for its guaranty of the bonds of the Blue Ridge Railroad Company, by providing for the securing and destruction of the same, approved March 2, 1872, providing for an annual tax of three mills on the dollar, for the redemption of the revenue bond scrip, be, and the same is hereby, repealed.",False,,None,NaN,NaN,NaN,NaN,NaN,0
2,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"Sec. 2. That so much of Section seventytwo (72), Chapter XII, Title III, as directs that the State Auditor shall, on or before November fifteenth, annually, give notice to each County Auditor of the rates per centum authorized by law to be levied for various State purposes, be, and the same is hereby, repealed, and the Comptroller General is hereby forbidden to levy any tax for any purpose whatever, unless expressly hereafter authorized so to do by statute. Approved October 22, 1873.",False,,None,NaN,2,NaN,NaN,NaN,2
3,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT to Revive, RENEw AND AMEND AN ACT ENTITLED “An Acr to Incorporate THE Home InsurANcE ComPANY, OF CHARLESTON.” Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the Act of the General Assembly of the said State, entitled “ An Act to incorporate the Home Insurance Company, of Charleston,” be, and the same is hereby, revived, renewed and extended, from the passage hereof, with the following alterations and amendments, viz: Section 5, strike out the words “one year” and insert in lieu thereof the words “ two years.” Section 8, after the word “ respondentia,” insert the words “or other securities.” Section 14, strike out the words “one year” and insert in lieu thereof “ two years.”",False,,2,NaN,NaN,NaN,NaN,NaN,0
4,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT To IncorPorRaTE THE REFORM Apotto Soclery, OF CHARLESTON, SouTH CAROLINA.",False,,3,NaN,NaN,NaN,NaN,NaN,0


<br>

In [47]:
map_dict = {
    'o':0,
    'O':0,
    'I':1,
    'i':1,
    'l':1,
    '!':1,
    'Z':2,
    'z':2,
    'S':5,
    'G':6
}

df_updated['section_comb'] = df_updated.apply(lambda x: fixCol(x['section_comb'], map_dict), axis =1)
df_updated['act'] = df_updated.apply(lambda x: fixCol(x['act'], map_dict), axis =1)

In [48]:
df_updated.head()

,year,first,start_page,end_page,path,sentence,flag,org_words,act,section_rgx1,section_rgx3,section_rgx4,section_rgx5,section_rgx6,section_comb
0,1873-1874,True,00000031,00000031,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"AN ACT to Repeat Section Four (4) or an AcT ENTITLED “An Act To RELIEVE THE STATE OF SOUTH CAROLINA OF ALL LIABILITY FOR ITS GUARANTY OF THE Bonps OF THE BLUE Ripce RaiLRoap Company, BY PROVIDING FOR THE SECURING AND DESTRUCTION OF THE SAME,” APPROVED Marcu 2, 1872; AND TO REPEAL sO MUCH OF SECTION 72, CHaprer XII, TitLE III, or THE GENERAL STATUTES, AS AUTHORIZES THE STATE AUDITOR TO GIVE Notice ANNUALLY TO Eacu County AvupITOR OF THE Rates PER CENTUM TO BE LEVIED FOR VARIOUS SratE PuRpPosEs.",False,,1,NaN,NaN,NaN,NaN,NaN,0
1,1873-1874,False,00000031,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"| Section 1. Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That Section four (4) of an Act entitled “An Act to relieve the State of South Carolina of all liability for its guaranty of the bonds of the Blue Ridge Railroad Company, by providing for the securing and destruction of the same, approved March 2, 1872, providing for an annual tax of three mills on the dollar, for the redemption of the revenue bond scrip, be, and the same is hereby, repealed.",False,,None,NaN,NaN,NaN,NaN,NaN,0
2,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"Sec. 2. That so much of Section seventytwo (72), Chapter XII, Title III, as directs that the State Auditor shall, on or before November fifteenth, annually, give notice to each County Auditor of the rates per centum authorized by law to be levied for various State purposes, be, and the same is hereby, repealed, and the Comptroller General is hereby forbidden to levy any tax for any purpose whatever, unless expressly hereafter authorized so to do by statute. Approved October 22, 1873.",False,,None,NaN,2,NaN,NaN,NaN,2
3,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT to Revive, RENEw AND AMEND AN ACT ENTITLED “An Acr to Incorporate THE Home InsurANcE ComPANY, OF CHARLESTON.” Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the Act of the General Assembly of the said State, entitled “ An Act to incorporate the Home Insurance Company, of Charleston,” be, and the same is hereby, revived, renewed and extended, from the passage hereof, with the following alterations and amendments, viz: Section 5, strike out the words “one year” and insert in lieu thereof the words “ two years.” Section 8, after the word “ respondentia,” insert the words “or other securities.” Section 14, strike out the words “one year” and insert in lieu thereof “ two years.”",False,,2,NaN,NaN,NaN,NaN,NaN,0
4,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT To IncorPorRaTE THE REFORM Apotto Soclery, OF CHARLESTON, SouTH CAROLINA.",False,,3,NaN,NaN,NaN,NaN,NaN,0


<br>

In [49]:
def fillMissing(value, firstIndex):
    """
    Fill in missing values for a column by setting missing values to the previous value.
    Not using ffill() from Pandas since we need to account for special cases (basically
    reseting the lastValid index) in the case that a new volume starts.
    Requires lastValid variable to be defined outside of the function.

    Parameters
    ----------
    value : str
        Value to check and fill.
    firstIndex : bool
        A flag for whether this row's value is the first value of a volume.
        Basically whether a new volume is starting.

    Returns
    -------
    str
        The new value.
    """

    global lastValid

    # If at the first index
    if firstIndex == True:
            
        # If a value doesn't exist, set that value and lastValid to 0
        if pd.isnull(value):
            value = 0
        
        # Set the lastValid's value to this value
        lastValid = value
        return value        


    # Else, if not at the first index
    # If a value does not exist
    if pd.isnull(value):
        # Set value to the lastValid value
        value = lastValid

    # If a value exists, that update lastValid
    else:
        lastValid = value

    return value

In [50]:
lastValid = 0
df_updated['section_comb'] = df_updated.apply(lambda x: fillMissing(x['section_comb'], x['first']), axis =1)

lastValid = 0
df_updated['act'] = df_updated.apply(lambda x: fillMissing(x['act'], x['first']), axis =1)

In [51]:
df_updated.head()

,year,first,start_page,end_page,path,sentence,flag,org_words,act,section_rgx1,section_rgx3,section_rgx4,section_rgx5,section_rgx6,section_comb
0,1873-1874,True,00000031,00000031,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"AN ACT to Repeat Section Four (4) or an AcT ENTITLED “An Act To RELIEVE THE STATE OF SOUTH CAROLINA OF ALL LIABILITY FOR ITS GUARANTY OF THE Bonps OF THE BLUE Ripce RaiLRoap Company, BY PROVIDING FOR THE SECURING AND DESTRUCTION OF THE SAME,” APPROVED Marcu 2, 1872; AND TO REPEAL sO MUCH OF SECTION 72, CHaprer XII, TitLE III, or THE GENERAL STATUTES, AS AUTHORIZES THE STATE AUDITOR TO GIVE Notice ANNUALLY TO Eacu County AvupITOR OF THE Rates PER CENTUM TO BE LEVIED FOR VARIOUS SratE PuRpPosEs.",False,,1,NaN,NaN,NaN,NaN,NaN,0
1,1873-1874,False,00000031,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"| Section 1. Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That Section four (4) of an Act entitled “An Act to relieve the State of South Carolina of all liability for its guaranty of the bonds of the Blue Ridge Railroad Company, by providing for the securing and destruction of the same, approved March 2, 1872, providing for an annual tax of three mills on the dollar, for the redemption of the revenue bond scrip, be, and the same is hereby, repealed.",False,,1,NaN,NaN,NaN,NaN,NaN,0
2,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"Sec. 2. That so much of Section seventytwo (72), Chapter XII, Title III, as directs that the State Auditor shall, on or before November fifteenth, annually, give notice to each County Auditor of the rates per centum authorized by law to be levied for various State purposes, be, and the same is hereby, repealed, and the Comptroller General is hereby forbidden to levy any tax for any purpose whatever, unless expressly hereafter authorized so to do by statute. Approved October 22, 1873.",False,,1,NaN,2,NaN,NaN,NaN,2
3,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT to Revive, RENEw AND AMEND AN ACT ENTITLED “An Acr to Incorporate THE Home InsurANcE ComPANY, OF CHARLESTON.” Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the Act of the General Assembly of the said State, entitled “ An Act to incorporate the Home Insurance Company, of Charleston,” be, and the same is hereby, revived, renewed and extended, from the passage hereof, with the following alterations and amendments, viz: Section 5, strike out the words “one year” and insert in lieu thereof the words “ two years.” Section 8, after the word “ respondentia,” insert the words “or other securities.” Section 14, strike out the words “one year” and insert in lieu thereof “ two years.”",False,,2,NaN,NaN,NaN,NaN,NaN,0
4,1873-1874,False,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT To IncorPorRaTE THE REFORM Apotto Soclery, OF CHARLESTON, SouTH CAROLINA.",False,,3,NaN,NaN,NaN,NaN,NaN,0


<br>

In [52]:
df_updated.rename({'section_comb':'section'}, axis=1, inplace=True)

In [53]:
cols_keep = ['year', 'sentence', 'start_page', 'end_page', 'act', 'section', 'path']
df_updated.drop(
    list(set(df_updated.columns.tolist()) - set(cols_keep)), axis = 1, inplace=True)

In [54]:
print(f"Length of the dataframe: {df_updated.shape[0]}")
display(df_updated)

Length of the dataframe: 39729


,year,start_page,end_page,path,sentence,act,section
0,1873-1874,00000031,00000031,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"AN ACT to Repeat Section Four (4) or an AcT ENTITLED “An Act To RELIEVE THE STATE OF SOUTH CAROLINA OF ALL LIABILITY FOR ITS GUARANTY OF THE Bonps OF THE BLUE Ripce RaiLRoap Company, BY PROVIDING FOR THE SECURING AND DESTRUCTION OF THE SAME,” APPROVED Marcu 2, 1872; AND TO REPEAL sO MUCH OF SECTION 72, CHaprer XII, TitLE III, or THE GENERAL STATUTES, AS AUTHORIZES THE STATE AUDITOR TO GIVE Notice ANNUALLY TO Eacu County AvupITOR OF THE Rates PER CENTUM TO BE LEVIED FOR VARIOUS SratE PuRpPosEs.",1,0
1,1873-1874,00000031,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"| Section 1. Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That Section four (4) of an Act entitled “An Act to relieve the State of South Carolina of all liability for its guaranty of the bonds of the Blue Ridge Railroad Company, by providing for the securing and destruction of the same, approved March 2, 1872, providing for an annual tax of three mills on the dollar, for the redemption of the revenue bond scrip, be, and the same is hereby, repealed.",1,0
2,1873-1874,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"Sec. 2. That so much of Section seventytwo (72), Chapter XII, Title III, as directs that the State Auditor shall, on or before November fifteenth, annually, give notice to each County Auditor of the rates per centum authorized by law to be levied for various State purposes, be, and the same is hereby, repealed, and the Comptroller General is hereby forbidden to levy any tax for any purpose whatever, unless expressly hereafter authorized so to do by statute. Approved October 22, 1873.",1,2
3,1873-1874,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT to Revive, RENEw AND AMEND AN ACT ENTITLED “An Acr to Incorporate THE Home InsurANcE ComPANY, OF CHARLESTON.” Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the Act of the General Assembly of the said State, entitled “ An Act to incorporate the Home Insurance Company, of Charleston,” be, and the same is hereby, revived, renewed and extended, from the passage hereof, with the following alterations and amendments, viz: Section 5, strike out the words “one year” and insert in lieu thereof the words “ two years.” Section 8, after the word “ respondentia,” insert the words “or other securities.” Section 14, strike out the words “one year” and insert in lieu thereof “ two years.”",2,0
4,1873-1874,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT To IncorPorRaTE THE REFORM Apotto Soclery, OF CHARLESTON, SouTH CAROLINA.",3,0
...,...,...,...,...,...,...,...
39724,1956,01448,01448,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1956/images/01448.jpg,"The Commissioner of Agriculture of South Carolina shall determine what are noxious weeds and plants, and shall publish such determination in suitable rules and regulations which shall be duly promulgated in accordance with law.",1132,1
39725,1956,01448,01448,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1956/images/01448.jpg,"Any such plants or weeds unlawfully imported into the State may be seized and confiscated, 

<br>

## Post-Cleaning
Some cleaning after adding Section and Acts.

In [55]:
def upperIfNeeded(sentence, ratio = 0.50):
    """
    Convert the given sentence list into an uppercase sentence list
    if the ratio of uppercase words (not including the ones with a mix of digits 
    or words like "SECTION") to the total words is greater than a fixed value.
    Needs an `uppered` variable to be defined outside of this scope.
    
    Parameters
    ----------
    sentence: str
         A str of sentence to check and convert to uppercase
        
    Returns
    -------
    str
        If check is approved the return an uppercase version of str.
        Else return the sentence.
    """

    global uppered
    
    # A count of the number of already uppercased words
    count = 0
    
    # Check whether the word consists of only letters,
    # has a length greater than 1, is uppercase, and 
    # isn't "SECTION"
    for word in sentence.split(" "):
        if word.isalpha() and len(word) > 1 and word.isupper() and word != "SECTION":
            count += 1

    # If the count to words ratio is greater
    # return all uppercase words
    if (count/len(sentence.split(" ")) > ratio):
        uppered += 1
        return sentence.upper()        
    
    # Else, return the original sentence list
    return sentence

In [56]:
uppered = 0
df_updated['sentence'] = df_updated.apply(lambda x: upperIfNeeded(x['sentence']), axis=1)
errorsDict['Uppercased'] = uppered

In [57]:
errorsDict

{'Split Sentences': 190,
 'Approved phrases': 2380,
 'EOL hyphenation': 42906,
 'Act seperators': 0,
 'Uppercased': 505}

<br>

## Character Length
Add the character length feature.
<br>This is added here because the lengths of the sentences might have changed during the cleaning process above.

In [58]:
df_updated["length"] = df_updated['sentence'].str.len()

In [59]:
df_updated.head()

,year,start_page,end_page,path,sentence,act,section,length
0,1873-1874,00000031,00000031,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"AN ACT TO REPEAT SECTION FOUR (4) OR AN ACT ENTITLED “AN ACT TO RELIEVE THE STATE OF SOUTH CAROLINA OF ALL LIABILITY FOR ITS GUARANTY OF THE BONPS OF THE BLUE RIPCE RAILROAP COMPANY, BY PROVIDING FOR THE SECURING AND DESTRUCTION OF THE SAME,” APPROVED MARCU 2, 1872; AND TO REPEAL SO MUCH OF SECTION 72, CHAPRER XII, TITLE III, OR THE GENERAL STATUTES, AS AUTHORIZES THE STATE AUDITOR TO GIVE NOTICE ANNUALLY TO EACU COUNTY AVUPITOR OF THE RATES PER CENTUM TO BE LEVIED FOR VARIOUS SRATE PURPPOSES.",1,0,503
1,1873-1874,00000031,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"| Section 1. Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That Section four (4) of an Act entitled “An Act to relieve the State of South Carolina of all liability for its guaranty of the bonds of the Blue Ridge Railroad Company, by providing for the securing and destruction of the same, approved March 2, 1872, providing for an annual tax of three mills on the dollar, for the redemption of the revenue bond scrip, be, and the same is hereby, repealed.",1,0,577
2,1873-1874,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"Sec. 2. That so much of Section seventytwo (72), Chapter XII, Title III, as directs that the State Auditor shall, on or before November fifteenth, annually, give notice to each County Auditor of the rates per centum authorized by law to be levied for various State purposes, be, and the same is hereby, repealed, and the Comptroller General is hereby forbidden to levy any tax for any purpose whatever, unless expressly hereafter authorized so to do by statute. Approved October 22, 1873.",1,2,493
3,1873-1874,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT to Revive, RENEw AND AMEND AN ACT ENTITLED “An Acr to Incorporate THE Home InsurANcE ComPANY, OF CHARLESTON.” Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the Act of the General Assembly of the said State, entitled “ An Act to incorporate the Home Insurance Company, of Charleston,” be, and the same is hereby, revived, renewed and extended, from the passage hereof, with the following alterations and amendments, viz: Section 5, strike out the words “one year” and insert in lieu thereof the words “ two years.” Section 8, after the word “ respondentia,” insert the words “or other securities.” Section 14, strike out the words “one year” and insert in lieu thereof “ two years.”",2,0,817
4,1873-1874,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT To IncorPorRaTE THE REFORM Apotto Soclery, OF CHARLESTON, SouTH CAROLINA.",3,0,85


<br>

### Removing Sentences With Low Character Length

Get rid of sentences with a low number of characters as they might not form meaningful sentences.
<br>Define a cutoff for the sentences. All sentences belows this length will be removed.
<br>Our research has shown that 30 character limit seems to be optimal for keeping informative sentences in the corpus.

In [60]:
cut_len = 25

In [61]:
# Initial length
ilen = df_updated.shape[0]

In [62]:
df_updated = df_updated[ df_updated["length"] > cut_len ]
print("Length of the cleaned dataframe: ", df_updated.shape[0])
print("Reduction of about {:.2f}%".format( (1 - df_updated.shape[0]/ilen) * 100))

Length of the cleaned dataframe:  31369
Reduction of about 21.04%


In [63]:
df_updated.reset_index(drop=True, inplace=True)

In [64]:
df_updated

,year,start_page,end_page,path,sentence,act,section,length
0,1873-1874,00000031,00000031,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"AN ACT TO REPEAT SECTION FOUR (4) OR AN ACT ENTITLED “AN ACT TO RELIEVE THE STATE OF SOUTH CAROLINA OF ALL LIABILITY FOR ITS GUARANTY OF THE BONPS OF THE BLUE RIPCE RAILROAP COMPANY, BY PROVIDING FOR THE SECURING AND DESTRUCTION OF THE SAME,” APPROVED MARCU 2, 1872; AND TO REPEAL SO MUCH OF SECTION 72, CHAPRER XII, TITLE III, OR THE GENERAL STATUTES, AS AUTHORIZES THE STATE AUDITOR TO GIVE NOTICE ANNUALLY TO EACU COUNTY AVUPITOR OF THE RATES PER CENTUM TO BE LEVIED FOR VARIOUS SRATE PURPPOSES.",1,0,503
1,1873-1874,00000031,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"| Section 1. Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That Section four (4) of an Act entitled “An Act to relieve the State of South Carolina of all liability for its guaranty of the bonds of the Blue Ridge Railroad Company, by providing for the securing and destruction of the same, approved March 2, 1872, providing for an annual tax of three mills on the dollar, for the redemption of the revenue bond scrip, be, and the same is hereby, repealed.",1,0,577
2,1873-1874,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"Sec. 2. That so much of Section seventytwo (72), Chapter XII, Title III, as directs that the State Auditor shall, on or before November fifteenth, annually, give notice to each County Auditor of the rates per centum authorized by law to be levied for various State purposes, be, and the same is hereby, repealed, and the Comptroller General is hereby forbidden to levy any tax for any purpose whatever, unless expressly hereafter authorized so to do by statute. Approved October 22, 1873.",1,2,493
3,1873-1874,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT to Revive, RENEw AND AMEND AN ACT ENTITLED “An Acr to Incorporate THE Home InsurANcE ComPANY, OF CHARLESTON.” Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the Act of the General Assembly of the said State, entitled “ An Act to incorporate the Home Insurance Company, of Charleston,” be, and the same is hereby, revived, renewed and extended, from the passage hereof, with the following alterations and amendments, viz: Section 5, strike out the words “one year” and insert in lieu thereof the words “ two years.” Section 8, after the word “ respondentia,” insert the words “or other securities.” Section 14, strike out the words “one year” and insert in lieu thereof “ two years.”",2,0,817
4,1873-1874,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT To IncorPorRaTE THE REFORM Apotto Soclery, OF CHARLESTON, SouTH CAROLINA.",3,0,85
...,...,...,...,...,...,...,...,...
31364,1956,01448,01448,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1956/images/01448.jpg,Be it enacted by the General Assembly of the State of South Carolina Commissioner of Agricultureregulate importation of noxious weedspenalties SECTION 1.,1132,0,158
31365,1956,01448,01448,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1956/images/01448.jpg,"The Commissioner of Agriculture of South Carolina shall determine what are noxious weeds and plants, and shall publish such determina

<br>

## Dropping duplicates

In [65]:
print(f"The number of dropped sentences is {df_updated[df_updated.duplicated(subset=['sentence'])].shape[0]}")

The number of dropped sentences is 1221


In [66]:
df_dropped = df_updated.drop_duplicates(subset=['sentence'])

In [67]:
print(f"Length of the dataframe: {df_dropped.shape[0]}")
display(df_dropped)

Length of the dataframe: 30148


,year,start_page,end_page,path,sentence,act,section,length
0,1873-1874,00000031,00000031,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"AN ACT TO REPEAT SECTION FOUR (4) OR AN ACT ENTITLED “AN ACT TO RELIEVE THE STATE OF SOUTH CAROLINA OF ALL LIABILITY FOR ITS GUARANTY OF THE BONPS OF THE BLUE RIPCE RAILROAP COMPANY, BY PROVIDING FOR THE SECURING AND DESTRUCTION OF THE SAME,” APPROVED MARCU 2, 1872; AND TO REPEAL SO MUCH OF SECTION 72, CHAPRER XII, TITLE III, OR THE GENERAL STATUTES, AS AUTHORIZES THE STATE AUDITOR TO GIVE NOTICE ANNUALLY TO EACU COUNTY AVUPITOR OF THE RATES PER CENTUM TO BE LEVIED FOR VARIOUS SRATE PURPPOSES.",1,0,503
1,1873-1874,00000031,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"| Section 1. Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That Section four (4) of an Act entitled “An Act to relieve the State of South Carolina of all liability for its guaranty of the bonds of the Blue Ridge Railroad Company, by providing for the securing and destruction of the same, approved March 2, 1872, providing for an annual tax of three mills on the dollar, for the redemption of the revenue bond scrip, be, and the same is hereby, repealed.",1,0,577
2,1873-1874,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"Sec. 2. That so much of Section seventytwo (72), Chapter XII, Title III, as directs that the State Auditor shall, on or before November fifteenth, annually, give notice to each County Auditor of the rates per centum authorized by law to be levied for various State purposes, be, and the same is hereby, repealed, and the Comptroller General is hereby forbidden to levy any tax for any purpose whatever, unless expressly hereafter authorized so to do by statute. Approved October 22, 1873.",1,2,493
3,1873-1874,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT to Revive, RENEw AND AMEND AN ACT ENTITLED “An Acr to Incorporate THE Home InsurANcE ComPANY, OF CHARLESTON.” Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the Act of the General Assembly of the said State, entitled “ An Act to incorporate the Home Insurance Company, of Charleston,” be, and the same is hereby, revived, renewed and extended, from the passage hereof, with the following alterations and amendments, viz: Section 5, strike out the words “one year” and insert in lieu thereof the words “ two years.” Section 8, after the word “ respondentia,” insert the words “or other securities.” Section 14, strike out the words “one year” and insert in lieu thereof “ two years.”",2,0,817
4,1873-1874,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT To IncorPorRaTE THE REFORM Apotto Soclery, OF CHARLESTON, SouTH CAROLINA.",3,0,85
...,...,...,...,...,...,...,...,...
31363,1956,01448,01448,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1956/images/01448.jpg,1132 An Act To Require The Commissioner Of Agriculture Of South Carolina To Determine What Are Noxious Weeds And Plants; To Prevent Importation Into The State Of Such Weeds And Plants And To Provide A Penalty For The Violation of The Provisions Of This Act.,1132,113,262
31364,1956,01448,01448,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1956/images/01448.jpg,Be it enacted by the General

<br>

## Adding Features

In [68]:
import warnings
warnings.filterwarnings("ignore")

<br>

### Adding ID

In [69]:
# The final dataframe (will be concatenated) containing the id's
df_final = []

# Loop through each year in the dataframe
for year in years:
    
    # Make a temporary dataframe that only contains rows for this year
    df_temp = df_dropped[df_dropped.year.str.startswith(year)]

    # Reset it's index
    df_temp.reset_index(drop=True, inplace=True)

    # Add a new column with the index value
    df_temp['id'] = df_temp.index.values
    
    # Get the length of the id of the last row in the dataframe, which is used to assess how many 0's will be prefixed to the other ids
    maxNumLength = len(str(df_temp.last_valid_index()))

    # Add the updated id
    df_temp['id'] = df_temp.apply(lambda x: str(year) + "_" + addPrefix( str(x['id']), maxNumLength ), axis=1)
        
    # Append to the final dataframe
    df_final.append(df_temp)

In [70]:
# Concatenate the final dataframe
df_final = pd.concat(df_final, ignore_index=True)

# Set it's index to the new ids
df_final.set_index('id', inplace=True)

In [71]:
print(f"Length of the dataframe: {df_final.shape[0]}")
display(df_final)

Length of the dataframe: 30148


,year,start_page,end_page,path,sentence,act,section,length
id,,,,,,,,
1873-1874_0000,1873-1874,00000031,00000031,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"AN ACT TO REPEAT SECTION FOUR (4) OR AN ACT ENTITLED “AN ACT TO RELIEVE THE STATE OF SOUTH CAROLINA OF ALL LIABILITY FOR ITS GUARANTY OF THE BONPS OF THE BLUE RIPCE RAILROAP COMPANY, BY PROVIDING FOR THE SECURING AND DESTRUCTION OF THE SAME,” APPROVED MARCU 2, 1872; AND TO REPEAL SO MUCH OF SECTION 72, CHAPRER XII, TITLE III, OR THE GENERAL STATUTES, AS AUTHORIZES THE STATE AUDITOR TO GIVE NOTICE ANNUALLY TO EACU COUNTY AVUPITOR OF THE RATES PER CENTUM TO BE LEVIED FOR VARIOUS SRATE PURPPOSES.",1,0,503
1873-1874_0001,1873-1874,00000031,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg,"| Section 1. Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That Section four (4) of an Act entitled “An Act to relieve the State of South Carolina of all liability for its guaranty of the bonds of the Blue Ridge Railroad Company, by providing for the securing and destruction of the same, approved March 2, 1872, providing for an annual tax of three mills on the dollar, for the redemption of the revenue bond scrip, be, and the same is hereby, repealed.",1,0,577
1873-1874_0002,1873-1874,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"Sec. 2. That so much of Section seventytwo (72), Chapter XII, Title III, as directs that the State Auditor shall, on or before November fifteenth, annually, give notice to each County Auditor of the rates per centum authorized by law to be levied for various State purposes, be, and the same is hereby, repealed, and the Comptroller General is hereby forbidden to levy any tax for any purpose whatever, unless expressly hereafter authorized so to do by statute. Approved October 22, 1873.",1,2,493
1873-1874_0003,1873-1874,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT to Revive, RENEw AND AMEND AN ACT ENTITLED “An Acr to Incorporate THE Home InsurANcE ComPANY, OF CHARLESTON.” Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the Act of the General Assembly of the said State, entitled “ An Act to incorporate the Home Insurance Company, of Charleston,” be, and the same is hereby, revived, renewed and extended, from the passage hereof, with the following alterations and amendments, viz: Section 5, strike out the words “one year” and insert in lieu thereof the words “ two years.” Section 8, after the word “ respondentia,” insert the words “or other securities.” Section 14, strike out the words “one year” and insert in lieu thereof “ two years.”",2,0,817
1873-1874_0004,1873-1874,00000032,00000032,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg,"AN ACT To IncorPorRaTE THE REFORM Apotto Soclery, OF CHARLESTON, SouTH CAROLINA.",3,0,85
...,...,...,...,...,...,...,...,...
1956_4532,1956,01448,01448,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1956/images/01448.jpg,1132 An Act To Require The Commissioner Of Agriculture Of South Carolina To Determine What Are Noxious Weeds And Plants; To Prevent Importation Into The State Of Such Weeds And Plants And To Provide A Penalty For The Violation of The Provisions Of This Act.,1132,113,262
1956_4533,1956,01448,01448,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks

<br>

### Adding the Remaining Features

In [72]:
df_final.insert(1, 'law_type', pd.NA)
df_final.insert(2, 'state', 'SOUTH CAROLINA')

In [73]:
df_final['law_type'] = df_final.apply(lambda x : addJoints(x['sentence']), axis=1)

In [74]:
def fixJoints(row_law_type, row_Act_label):
    """
    Fix incorrect labels which should be "Joint Resolution".
    
    Parameters
    ----------
    row_law_type : str
        The current row's law type.
        Ex. 'Act' or 'Joint Resolution'

    row_Act_label: str
       The current row's Act label/value.

    Returns
    -------
    str
        Either "Joint Resolution" or `row_law_type`.
    
    """
    global joint_label

    # If the row's law_type is 'Joint Resolution', then assign the label number to `joint_label`.
    # This row is already a 'Joint Resolution'
    if row_law_type == 'Joint Resolution':
        joint_label = row_Act_label
        return 'Joint Resolution'
    
    # If this row is not a 'Joint Resolution', then
    # If the row's Act label is the same as `joint_label` then that row should be a Joint Resolution.
    elif row_Act_label == joint_label:
        return 'Joint Resolution'
    
    # Otherwise, that row is an act
    else:
        return row_law_type
        
        
joint_label = -1
df_final['law_type'] = df_final.apply(lambda x: fixJoints(x['law_type'], x['act']), axis = 1)

<br>

## Some Final Touches

In [75]:
df_final.drop('year', axis=1, inplace=True)

In [76]:
cols_keep.remove('year')
cols_keep.insert(0, 'state')
cols_keep.insert(0, 'law_type')
cols_keep.insert(3, 'length')

In [77]:
cols_keep

['law_type',
 'state',
 'sentence',
 'length',
 'start_page',
 'end_page',
 'act',
 'section',
 'path']

In [78]:
df_final = df_final[cols_keep]

In [79]:
print(f"Length of the dataframe: {df_final.shape[0]}")
display(df_final)

Length of the dataframe: 30148


,law_type,state,sentence,length,start_page,end_page,act,section,path
id,,,,,,,,,
1873-1874_0000,Act,SOUTH CAROLINA,"AN ACT TO REPEAT SECTION FOUR (4) OR AN ACT ENTITLED “AN ACT TO RELIEVE THE STATE OF SOUTH CAROLINA OF ALL LIABILITY FOR ITS GUARANTY OF THE BONPS OF THE BLUE RIPCE RAILROAP COMPANY, BY PROVIDING FOR THE SECURING AND DESTRUCTION OF THE SAME,” APPROVED MARCU 2, 1872; AND TO REPEAL SO MUCH OF SECTION 72, CHAPRER XII, TITLE III, OR THE GENERAL STATUTES, AS AUTHORIZES THE STATE AUDITOR TO GIVE NOTICE ANNUALLY TO EACU COUNTY AVUPITOR OF THE RATES PER CENTUM TO BE LEVIED FOR VARIOUS SRATE PURPPOSES.",503,00000031,00000031,1,0,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg
1873-1874_0001,Act,SOUTH CAROLINA,"| Section 1. Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That Section four (4) of an Act entitled “An Act to relieve the State of South Carolina of all liability for its guaranty of the bonds of the Blue Ridge Railroad Company, by providing for the securing and destruction of the same, approved March 2, 1872, providing for an annual tax of three mills on the dollar, for the redemption of the revenue bond scrip, be, and the same is hereby, repealed.",577,00000031,00000032,1,0,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000031.jpg
1873-1874_0002,Act,SOUTH CAROLINA,"Sec. 2. That so much of Section seventytwo (72), Chapter XII, Title III, as directs that the State Auditor shall, on or before November fifteenth, annually, give notice to each County Auditor of the rates per centum authorized by law to be levied for various State purposes, be, and the same is hereby, repealed, and the Comptroller General is hereby forbidden to levy any tax for any purpose whatever, unless expressly hereafter authorized so to do by statute. Approved October 22, 1873.",493,00000032,00000032,1,2,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg
1873-1874_0003,Act,SOUTH CAROLINA,"AN ACT to Revive, RENEw AND AMEND AN ACT ENTITLED “An Acr to Incorporate THE Home InsurANcE ComPANY, OF CHARLESTON.” Be it enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the Act of the General Assembly of the said State, entitled “ An Act to incorporate the Home Insurance Company, of Charleston,” be, and the same is hereby, revived, renewed and extended, from the passage hereof, with the following alterations and amendments, viz: Section 5, strike out the words “one year” and insert in lieu thereof the words “ two years.” Section 8, after the word “ respondentia,” insert the words “or other securities.” Section 14, strike out the words “one year” and insert in lieu thereof “ two years.”",817,00000032,00000032,2,0,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg
1873-1874_0004,Act,SOUTH CAROLINA,"AN ACT To IncorPorRaTE THE REFORM Apotto Soclery, OF CHARLESTON, SouTH CAROLINA.",85,00000032,00000032,3,0,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1873-1874/images.zip/00000032.jpg
...,...,...,...,...,...,...,...,...,...
1956_4532,Act,SOUTH CAROLINA,1132 An Act To Require The Commissioner Of Agriculture Of South Carolina To Determine What Are Noxious Weeds And Plants; To Prevent Importation Into The State Of Such Weeds And Plants And To Provide A Penalty For The Violation of The Provisions Of This Act.,262,01448,01448,1132,113,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1956/images/01448.jpg
1956_4533,Act,SOUT

<br>

## Exporting

In [80]:
# df_final.to_csv('final_splits_testing.csv')

In [81]:
# df_test = df_final.drop(['law_type', 'state', 'length', 'end_page', 'path'], axis=1)
# df_test.to_csv(f'{year}_testing.csv')
# print(f'exporting {year}')